## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
########################### 1-1. recursive model by store ##########################
####################################################################################

In [4]:
ver, KKK = 'priv', 0
STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
CLUSTERS_IDS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

In [5]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [6]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [7]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_cluster(cluster):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[df['tskm_10']==cluster]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3

    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for cluster_id in CLUSTERS_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_item_cluster14_'+str(cluster_id)+'.pkl')
        temp_df['cluster_id'] = cluster_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [8]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                } 


In [9]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','state_id','store_id','cat_id', 'dept_id', 'item_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = [#'enc_cat_id_mean','enc_cat_id_std',
                   #'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [10]:
########################### Train Models
#################################################################################
for cluster_id in CLUSTERS_IDS:
    print('Train cluster', str(cluster_id))
    
    grid_df, features_columns = get_data_by_cluster(cluster_id)
    
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)
    
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    
    d_sales = grid_df[['d','sales']]
    substitute = d_sales['sales'].values
    substitute[(d_sales['d'] > END_TRAIN)] = np.nan
    grid_df['sales'] = substitute
    
    grid_df.to_pickle(processed_data_dir+'test_item_cluster14_'+str(cluster_id)+'.pkl')
    del grid_df, d_sales, substitute
    

    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    display(pd.DataFrame({'name':estimator.feature_name(),
                          'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))

    
    model_name = model_dir+'lgb_model_item_cluster14'+str(cluster_id)+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    del train_data, valid_data, estimator
    gc.collect()
    
    MODEL_FEATURES = features_columns

Train cluster_ 0
[100]	valid_0's rmse: 1.20587
[200]	valid_0's rmse: 1.1912
[300]	valid_0's rmse: 1.18688
[400]	valid_0's rmse: 1.18412
[500]	valid_0's rmse: 1.18196
[600]	valid_0's rmse: 1.18014
[700]	valid_0's rmse: 1.17843
[800]	valid_0's rmse: 1.17689
[900]	valid_0's rmse: 1.17542
[1000]	valid_0's rmse: 1.17402
[1100]	valid_0's rmse: 1.17266
[1200]	valid_0's rmse: 1.17139
[1300]	valid_0's rmse: 1.17009
[1400]	valid_0's rmse: 1.16879
[1500]	valid_0's rmse: 1.16757
[1600]	valid_0's rmse: 1.16643
[1700]	valid_0's rmse: 1.16525
[1800]	valid_0's rmse: 1.16414
[1900]	valid_0's rmse: 1.16294
[2000]	valid_0's rmse: 1.16183
[2100]	valid_0's rmse: 1.16065
[2200]	valid_0's rmse: 1.15962
[2300]	valid_0's rmse: 1.15853
[2400]	valid_0's rmse: 1.15747
[2500]	valid_0's rmse: 1.15633
[2600]	valid_0's rmse: 1.15522
[2700]	valid_0's rmse: 1.15412
[2800]	valid_0's rmse: 1.15304
[2900]	valid_0's rmse: 1.15202
[3000]	valid_0's rmse: 1.15093


,name,imp
9,item_nunique,30809
53,rolling_std_180,29798
52,rolling_mean_180,29578
28,enc_item_id_std,26617
1,release,26531
27,enc_item_id_mean,26369
21,tm_w,26044
51,rolling_std_60,25004
20,tm_d,24228
50,rolling_mean_60,21886


Train cluster_ 1
[100]	valid_0's rmse: 6.63439
[200]	valid_0's rmse: 5.84097
[300]	valid_0's rmse: 5.81717
[400]	valid_0's rmse: 5.8012
[500]	valid_0's rmse: 5.78451
[600]	valid_0's rmse: 5.76553
[700]	valid_0's rmse: 5.75211
[800]	valid_0's rmse: 5.73903
[900]	valid_0's rmse: 5.73166
[1000]	valid_0's rmse: 5.72036
[1100]	valid_0's rmse: 5.70604
[1200]	valid_0's rmse: 5.70029
[1300]	valid_0's rmse: 5.69106
[1400]	valid_0's rmse: 5.68456
[1500]	valid_0's rmse: 5.67663
[1600]	valid_0's rmse: 5.66984
[1700]	valid_0's rmse: 5.66362
[1800]	valid_0's rmse: 5.65997
[1900]	valid_0's rmse: 5.65516
[2000]	valid_0's rmse: 5.6499
[2100]	valid_0's rmse: 5.64529
[2200]	valid_0's rmse: 5.64024
[2300]	valid_0's rmse: 5.63607
[2400]	valid_0's rmse: 5.63248
[2500]	valid_0's rmse: 5.62622
[2600]	valid_0's rmse: 5.62035
[2700]	valid_0's rmse: 5.61602
[2800]	valid_0's rmse: 5.61149
[2900]	valid_0's rmse: 5.60918
[3000]	valid_0's rmse: 5.60457


,name,imp
54,rolling_mean_tmp_1_7,1858
58,rolling_mean_tmp_7_7,1347
21,tm_w,1329
20,tm_d,1248
9,item_nunique,1184
25,tm_dw,955
52,rolling_mean_180,784
51,rolling_std_60,771
53,rolling_std_180,764
55,rolling_mean_tmp_1_14,757


Train cluster_ 2
[100]	valid_0's rmse: 11.4091
[200]	valid_0's rmse: 8.16169
[300]	valid_0's rmse: 7.98256
[400]	valid_0's rmse: 7.99111
[500]	valid_0's rmse: 8.0086
[600]	valid_0's rmse: 8.01278
[700]	valid_0's rmse: 8.02045
[800]	valid_0's rmse: 8.02724
[900]	valid_0's rmse: 8.03126
[1000]	valid_0's rmse: 8.03512
[1100]	valid_0's rmse: 8.03282
[1200]	valid_0's rmse: 8.03613
[1300]	valid_0's rmse: 8.03594
[1400]	valid_0's rmse: 8.03428
[1500]	valid_0's rmse: 8.03261
[1600]	valid_0's rmse: 8.03163
[1700]	valid_0's rmse: 8.03012
[1800]	valid_0's rmse: 8.02625
[1900]	valid_0's rmse: 8.02542
[2000]	valid_0's rmse: 8.02202
[2100]	valid_0's rmse: 8.02514
[2200]	valid_0's rmse: 8.02272
[2300]	valid_0's rmse: 8.02019
[2400]	valid_0's rmse: 8.01531
[2500]	valid_0's rmse: 8.01585
[2600]	valid_0's rmse: 8.01552
[2700]	valid_0's rmse: 8.0084
[2800]	valid_0's rmse: 8.0091
[2900]	valid_0's rmse: 8.00721
[3000]	valid_0's rmse: 8.00491


...

,name,imp
53,rolling_std_180,151932
52,rolling_mean_180,145513
9,item_nunique,145161
28,enc_item_id_std,135153
27,enc_item_id_mean,133405
21,tm_w,133059
1,release,127549
51,rolling_std_60,119118
20,tm_d,118282
5,price_std,104890


Train cluster_ 12
[100]	valid_0's rmse: 1.5694
[200]	valid_0's rmse: 1.53042
[300]	valid_0's rmse: 1.51816
[400]	valid_0's rmse: 1.51196
[500]	valid_0's rmse: 1.50776
[600]	valid_0's rmse: 1.50407
[700]	valid_0's rmse: 1.50095
[800]	valid_0's rmse: 1.4981
[900]	valid_0's rmse: 1.49571
[1000]	valid_0's rmse: 1.49352
[1100]	valid_0's rmse: 1.49139
[1200]	valid_0's rmse: 1.48948
[1300]	valid_0's rmse: 1.4876
[1400]	valid_0's rmse: 1.48578
[1500]	valid_0's rmse: 1.48401
[1600]	valid_0's rmse: 1.4824
[1700]	valid_0's rmse: 1.48081
[1800]	valid_0's rmse: 1.47923
[1900]	valid_0's rmse: 1.47779
[2000]	valid_0's rmse: 1.4763
[2100]	valid_0's rmse: 1.47492
[2200]	valid_0's rmse: 1.47353
[2300]	valid_0's rmse: 1.47198
[2400]	valid_0's rmse: 1.47062
[2500]	valid_0's rmse: 1.46921
[2600]	valid_0's rmse: 1.46785
[2700]	valid_0's rmse: 1.46638
[2800]	valid_0's rmse: 1.46507
[2900]	valid_0's rmse: 1.46375
[3000]	valid_0's rmse: 1.46252


,name,imp
9,item_nunique,19098
53,rolling_std_180,19020
52,rolling_mean_180,18325
28,enc_item_id_std,17223
1,release,16562
21,tm_w,16488
27,enc_item_id_mean,15335
20,tm_d,15228
51,rolling_std_60,15227
54,rolling_mean_tmp_1_7,14397


Train cluster_ 13
[100]	valid_0's rmse: 1.23263
[200]	valid_0's rmse: 1.22213
[300]	valid_0's rmse: 1.21906
[400]	valid_0's rmse: 1.21663
[500]	valid_0's rmse: 1.21445
[600]	valid_0's rmse: 1.21258
[700]	valid_0's rmse: 1.21087
[800]	valid_0's rmse: 1.20936
[900]	valid_0's rmse: 1.20787
[1000]	valid_0's rmse: 1.20644
[1100]	valid_0's rmse: 1.20508
[1200]	valid_0's rmse: 1.2037
[1300]	valid_0's rmse: 1.20233
[1400]	valid_0's rmse: 1.20103
[1500]	valid_0's rmse: 1.19978
[1600]	valid_0's rmse: 1.19857
[1700]	valid_0's rmse: 1.19732
[1800]	valid_0's rmse: 1.19611
[1900]	valid_0's rmse: 1.19486
[2000]	valid_0's rmse: 1.19363
[2100]	valid_0's rmse: 1.19241
[2200]	valid_0's rmse: 1.19121
[2300]	valid_0's rmse: 1.19002
[2400]	valid_0's rmse: 1.18883
[2500]	valid_0's rmse: 1.1877
[2600]	valid_0's rmse: 1.18653
[2700]	valid_0's rmse: 1.1853
[2800]	valid_0's rmse: 1.18414
[2900]	valid_0's rmse: 1.18297
[3000]	valid_0's rmse: 1.18189


,name,imp
53,rolling_std_180,69170
9,item_nunique,66761
52,rolling_mean_180,66323
28,enc_item_id_std,62673
27,enc_item_id_mean,60254
5,price_std,60098
21,tm_w,58817
20,tm_d,55703
51,rolling_std_60,54347
11,price_momentum_m,50969
